In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import re

In [ ]:
trainOrig = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
trainOrig['type'] = 'train'
test['type'] = 'val'

appended = pd.concat([trainOrig,test], axis = 0)

In [ ]:
appended.PassengerId.duplicated().all()

In [ ]:
# work with data
def manipulateData(train):
    # replace all the Nan  (Age, cabin, embarked)
    train['Age'].isnull().values.any()
    ageMean = train.Age.mean()
    train['Age'] = train.Age.fillna(0)
    train['Age'] = train.Age.replace(0, ageMean)
    train['Age'] = train['Age'].astype(int)
    
#     fareMean = train.Fare.mean()
#     train['Fare'] = train.Fare.fillna(0)
#     train['Fare'] = train.Fare.replace(0, fareMean)
#     train['Fare'] = train['Fare'].astype(int)
    train['Fare'] = train['Fare'].fillna(train['Fare'].median())
    train.loc[ train['Fare'] <= 7.91, 'Fare'] 						        = 0
    train.loc[(train['Fare'] > 7.91) & (train['Fare'] <= 14.454), 'Fare'] = 1
    train.loc[(train['Fare'] > 14.454) & (train['Fare'] <= 31), 'Fare']   = 2
    train.loc[ train['Fare'] > 31, 'Fare'] 							        = 3
    train['Fare'] = train['Fare'].astype(int)
    
    # Mapping Age
    train.loc[ train['Age'] <= 16, 'Age'] 					       = 0
    train.loc[(train['Age'] > 16) & (train['Age'] <= 32), 'Age'] = 1
    train.loc[(train['Age'] > 32) & (train['Age'] <= 48), 'Age'] = 2
    train.loc[(train['Age'] > 48) & (train['Age'] <= 64), 'Age'] = 3
    train.loc[ train['Age'] > 64, 'Age'] = 4 ;

    # S = 0, C = 1, Q = 2
    train['Embarked'] = train.Embarked.replace('S', 0)
    train['Embarked'] = train.Embarked.replace('C', 1)
    train['Embarked'] = train.Embarked.replace('Q', 2)
    print(train.Embarked.mean())
    train['Embarked'] = train.Embarked.fillna(train.Embarked.mean())

    # encode the cabin and ticket
    train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
    i = -1
    for cabin in train.Cabin.unique():
        train['Cabin'] = train.Cabin.replace(cabin, i)
        i = i + 1

    peopleInCabin =[]
    for cabin in train.Cabin:
        if cabin >= 0:
            peopleInCabin.append(train.loc[train.Cabin == cabin].shape[0])
        else:
            peopleInCabin.append(0)
    train['people_in_cabin'] = peopleInCabin

    i = 0
    for ticket in train.Ticket.unique():
        train['Ticket'] = train.Ticket.replace(ticket, i)
        i = i + 1 
    sameTicket =[]
    for ticket in train.Ticket:
        if ticket >= 0:
            sameTicket.append(train.loc[train.Ticket == ticket].shape[0])
        else:
            sameTicket.append(0)
    train['same_ticket'] = sameTicket

    #encode the gender
    train['Sex'] = train.Sex.replace('male', 0)
    train['Sex'] = train.Sex.replace('female', 1)

    #conde Mr, Mrs, Miss and last names
    i = 0
    lastName = []
    title = []
    for name in train['Name'].to_numpy():
        foundLastName = False
        foundM = False
        foundTitle = False
        indexM = 0
        j = 0
        for letter in name:
            if letter == ',':
                #train['LastName'] = name[0:j]
                lastName.append(name[0:j])
                foundLastName = True
            if foundLastName and (letter == 'M' or letter == 'D' or letter == 'R' or letter == 'L' or letter == 'S' or letter == 'C' or letter == 't' or letter == 'J'):
                foundM = True
                indexM = j
            if foundM and letter == '.':
                if foundTitle:
                    print(name)
                else:
                    title.append(name[indexM:j])
                foundTitle = True
            j = j+1
        if not foundTitle:
            print(name)
        i = i+1

    def get_title(name):
        title_search = re.search(' ([A-Za-z]+)\.', name)
        # If the title exists, extract and return it.
        if title_search:
            return title_search.group(1)
        return ""
    # Create a new feature Title, containing the titles of passenger names
    train['Title'] = train['Name'].apply(get_title)
    # Group all non-common titles into one single grouping "Rare"
    train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    train['Title'] = train['Title'].replace('Mlle', 'Miss')
    train['Title'] = train['Title'].replace('Ms', 'Miss')
    train['Title'] = train['Title'].replace('Mme', 'Mrs')
        
    #train['Title'] = title
    train['LastName'] = lastName


    i = 0
    for LastName in train.LastName.unique():
        train['LastName'] = train.LastName.replace(LastName, i)
        i = i + 1 

    sameLastName =[]
    for LastName in train.LastName:
        if LastName >= 0:
            sameLastName.append(train.loc[train.LastName == LastName].shape[0])
        else:
            sameLastName.append(0)
    train['same_last_name'] = sameLastName
    
    train['familySize'] = train.SibSp + train.Parch
    
    isAlone =[]
    for familySize in train.familySize:
        if familySize == 0:
            isAlone.append(1)
        else:
            isAlone.append(0)
    train['isAlone'] = isAlone
    
    i = 0
    for Title in train.Title.unique():
        train['Title'] = train.Title.replace(Title, i)
        i = i + 1 
    
    # encode the fare and the ticket (normalize)
    #train['Fare'] = (train.Fare - train.Fare.mean()) / (train.Fare.max()-train.Fare.min())
    #train['Ticket'] = (train.Ticket - train.Ticket.mean()) / (train.Ticket.max()-train.Ticket.min())
    #train['LastName'] = (train.LastName - train.LastName.mean()) / (train.LastName.max()-train.LastName.min())
    #train['Age'] = (train.Age - train.Age.mean()) / (train.Age.max()-train.Age.min())

    train = train.drop(['Name'], axis = 1)
    train = train.drop(['PassengerId'], axis = 1)
    train = train.drop(['Cabin'], axis = 1)
    train = train.drop(['Ticket'], axis = 1)
    train = train.drop(['SibSp'], axis = 1)
    train = train.drop(['Parch'], axis = 1)
    train = train.drop(['LastName'], axis = 1)
    
    return train

In [ ]:
manipulatedData = manipulateData(appended)

In [ ]:
manipulatedData

In [ ]:
survivedAge = manipulatedData['Age'].loc[manipulatedData.Survived == 1]
diedAge = manipulatedData['Age'].loc[manipulatedData.Survived == 0]

survivedAge.hist(bins = 100)

In [ ]:
diedAge.hist(bins = 100)

In [ ]:
training_data_manipulated = manipulatedData.loc[manipulatedData.type == 'train']
X_train = training_data_manipulated.drop(['Survived', 'type'], axis = 1)
Y_train = training_data_manipulated.Survived

val_data_manipulated = manipulatedData.loc[manipulatedData.type == 'val']
X_val = val_data_manipulated.drop(['Survived','type'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

# baseline model (random forest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)#, max_depth=10, n_estimators = 1000)
clf.fit(X_train, y_train)

print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

In [ ]:
clf.feature_importances_

# grid search

In [ ]:
param_grid = { 
    'n_estimators': [ 200,300,400,500],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=0), param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train,y_train)
CV_rfc.best_params_

In [ ]:
rfc1=RandomForestClassifier(random_state=0, n_estimators= 500, criterion = 'gini',max_features = 'auto',max_depth = 10)
rfc1.fit(X_train, y_train)
print(rfc1.score(X_train,y_train))
print(rfc1.score(X_test,y_test))

In [ ]:
#LastName
features = ["Pclass", "Sex", "SibSp", "Parch"]
LastName = RandomForestClassifier(random_state=0)
LastName.fit(X_train[features], y_train)
print(LastName.score(X_train[features],y_train))
print(LastName.score(X_test[features],y_test))

In [ ]:
pred = clf.predict(X_test)
y_test_np = y_test.to_numpy()
indexes = []
for i in range(0,len(pred)):
    if(pred[i] !=y_test_np[i]):
        indexes.append(i)
        print('pred', pred[i], ' truth', y_test_np[i])
        #print(X_test.iloc[i])
print(indexes)

# random forest on diff features

In [ ]:
#class
cass = RandomForestClassifier(random_state=0)
cass.fit(np.reshape(X_train['Pclass'].to_numpy(),(X_train.shape[0],1)), y_train)
print(cass.score(np.reshape(X_train['Pclass'].to_numpy(),(X_train.shape[0],1)),y_train))
print(cass.score(np.reshape(X_test['Pclass'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#sex
sex = RandomForestClassifier(random_state=0)
sex.fit(np.reshape(X_train['Sex'].to_numpy(),(X_train.shape[0],1)), y_train)
print(sex.score(np.reshape(X_train['Sex'].to_numpy(),(X_train.shape[0],1)),y_train))
print(sex.score(np.reshape(X_test['Sex'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#age
age = RandomForestClassifier(random_state=0)
age.fit(np.reshape(X_train['Age'].to_numpy(),(X_train.shape[0],1)), y_train)
print(age.score(np.reshape(X_train['Age'].to_numpy(),(X_train.shape[0],1)),y_train))
print(age.score(np.reshape(X_test['Age'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#SibSp
SibSp = RandomForestClassifier(random_state=0)
SibSp.fit(np.reshape(X_train['SibSp'].to_numpy(),(X_train.shape[0],1)), y_train)
print(SibSp.score(np.reshape(X_train['SibSp'].to_numpy(),(X_train.shape[0],1)),y_train))
print(SibSp.score(np.reshape(X_test['SibSp'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#Parch
Parch = RandomForestClassifier(random_state=0)
Parch.fit(np.reshape(X_train['Parch'].to_numpy(),(X_train.shape[0],1)), y_train)
print(Parch.score(np.reshape(X_train['Parch'].to_numpy(),(X_train.shape[0],1)),y_train))
print(Parch.score(np.reshape(X_test['Parch'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#Ticket
Ticket = RandomForestClassifier(random_state=0)
Ticket.fit(np.reshape(X_train['Ticket'].to_numpy(),(X_train.shape[0],1)), y_train)
print(Ticket.score(np.reshape(X_train['Ticket'].to_numpy(),(X_train.shape[0],1)),y_train))
print(Ticket.score(np.reshape(X_test['Ticket'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#Fare
Fare = RandomForestClassifier(random_state=0)
Fare.fit(np.reshape(X_train['Fare'].to_numpy(),(X_train.shape[0],1)), y_train)
print(Fare.score(np.reshape(X_train['Fare'].to_numpy(),(X_train.shape[0],1)),y_train))
print(Fare.score(np.reshape(X_test['Fare'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#Cabin
Cabin = RandomForestClassifier(random_state=0)
Cabin.fit(np.reshape(X_train['Cabin'].to_numpy(),(X_train.shape[0],1)), y_train)
print(Cabin.score(np.reshape(X_train['Cabin'].to_numpy(),(X_train.shape[0],1)),y_train))
print(Cabin.score(np.reshape(X_test['Cabin'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#Embarked
Embarked = RandomForestClassifier(random_state=0)
Embarked.fit(np.reshape(X_train['Embarked'].to_numpy(),(X_train.shape[0],1)), y_train)
print(Embarked.score(np.reshape(X_train['Embarked'].to_numpy(),(X_train.shape[0],1)),y_train))
print(Embarked.score(np.reshape(X_test['Embarked'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#Title
Title = RandomForestClassifier(random_state=0)
Title.fit(np.reshape(X_train['Title'].to_numpy(),(X_train.shape[0],1)), y_train)
print(Title.score(np.reshape(X_train['Title'].to_numpy(),(X_train.shape[0],1)),y_train))
print(Title.score(np.reshape(X_test['Title'].to_numpy(),(X_test.shape[0],1)),y_test))

In [ ]:
#LastName
LastName = RandomForestClassifier(random_state=0)
LastName.fit(np.reshape(X_train['LastName'].to_numpy(),(X_train.shape[0],1)), y_train)
print(LastName.score(np.reshape(X_train['LastName'].to_numpy(),(X_train.shape[0],1)),y_train))
print(LastName.score(np.reshape(X_test['LastName'].to_numpy(),(X_test.shape[0],1)),y_test))

# neural network

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
n_inputFeatures = X_train.shape[1]
activation = "relu"
size = 100
X_input = Input(shape=(n_inputFeatures,))

X = Dense(10, activation=activation)(X_input)
#X = Dense(60, activation=activation)(X)
# X = Dense(20, activation=activation)(X)

X = Dense(1, activation="sigmoid")(X)

# Create model
nn = Model(inputs = X_input, outputs = X, name='deepNN')

optAdam = tf.keras.optimizers.Adam()
nn.compile(optimizer=optAdam, loss='binary_crossentropy', metrics='binary_accuracy')

In [ ]:
history = nn.fit(X_train, y_train, epochs = 500, batch_size = 100000, validation_data=(X_test, y_test))

In [ ]:
#30 - 30 nn for 1000 epochs (overfitted at around 400, best test acc = 0.8475) loss: 0.2809 - binary_accuracy: 0.8742 - val_loss: 0.4978 - val_binary_accuracy: 0.8305
#10 -10 nn for 1000 epochs (overfitted around 600, best acc = 0.8169)
#100 -100 nn for 1000 epochs (overfitted around 150, best acc = 0.8407)
# 60-30-10 nn for 1000 epochs (overfitted around 250, best acc = 0.8339)
training_loss = history.history['loss']
val_loss      = history.history['val_loss']
epoch_count = range(1, len(training_loss) + 1)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, val_loss, 'g--')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# boosted tree


In [ ]:
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb

In [ ]:
evalset = [(X_train, y_train), (X_test, y_test)]
boostedTree = XGBClassifier(learning_rate=0.1,# n_estimators = 100)
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1)
boostedTree.fit(X_train, y_train,  eval_set=evalset, early_stopping_rounds=50)

print(boostedTree.score(X_train, y_train))
print(boostedTree.score(X_test, y_test))
#learning rate 0.1, best logloss = 0.425 (30 steps, 5 steps early stopping)

# k means

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0, verbose = 1, n_init = 2, tol = 1e-26, precompute_distances = True).fit(X_train)

# logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression(random_state=0).fit(X_train, y_train)
print(logReg.score(X_train,y_train))
print(logReg.score(X_test,y_test))

# stacked models

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier

estimators = [('rf1', RandomForestClassifier(n_estimators = 100,max_depth=10, random_state=42)),
              ('rf2', RandomForestClassifier(n_estimators = 100,max_depth=2, random_state=42)),
              ('rf3', RandomForestClassifier(n_estimators = 100,max_depth=6, random_state=42)),
              ('rf4', RandomForestClassifier(n_estimators = 100,max_depth=4, random_state=42)),
              ('xgboost1', XGBClassifier(objective= 'binary:logistic',learning_rate=0.1, max_depth=10, random_state=42)),
              ('xgboost2', XGBClassifier(objective= 'binary:logistic',learning_rate=0.1, max_depth=8, random_state=42)),
              ('xgboost3', XGBClassifier(objective= 'binary:logistic',learning_rate=0.1, max_depth=6, random_state=42)),
              ('xgboost4', XGBClassifier(objective= 'binary:logistic',learning_rate=0.1, max_depth=4, random_state=42)),
              #('svr', make_pipeline(StandardScaler(),LinearSVC(random_state=42))),
              ('nn1', MLPClassifier(hidden_layer_sizes = (10,),random_state=42, max_iter=500, early_stopping=True)),
              ('nn2', MLPClassifier(hidden_layer_sizes = (50,),random_state=42, max_iter=500, early_stopping=True)),
              ('nn3', MLPClassifier(hidden_layer_sizes = (50, 30,10,),random_state=42, max_iter=500, early_stopping=True)),
              ('logReg', LogisticRegression()),
            # ('kmeans', make_pipeline(StandardScaler(),KMeans(random_state=42))),
              
             ]

clf = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression()
 )

In [ ]:
clf.fit(X_train, y_train)
print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))